In [36]:
import json
import sqlite3

import chromadb
import keras
import numpy as np
from chromadb.utils import embedding_functions
from keras_tuner import RandomSearch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [37]:
TRAIN_CUTOFF = 3003  # 2711 2424 2155 1880 1572 1016 817 502 260

In [38]:
db = "../data/arxiv_papers.db"
# last_interested = find_last_interested_entry(db)
# print(json.dumps(last_interested, indent=4))

In [39]:
def get_data(database_path):
    # Create a database connection
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.row_factory = sqlite3.Row

    # SQL query to find the last occurrence of "interested" = 1
    query = f"""
    SELECT paper_id, concise_summary, interested FROM papers 
    ORDER BY paper_id ASC 
    LIMIT {TRAIN_CUTOFF};
    """

    try:
        cursor.execute(query)
        articles = cursor.fetchall()
        if articles:
            print(f"Got {len(articles)}.")
            return articles
        else:
            print("No interested entries found.")
            return nil
    except sqlite3.Error as e:
        print("Database error:", e)
    finally:
        conn.close()

In [40]:
def get_embedding(paper_id, vdb_path="../data/arxiv_embeddings.chroma"):
    vdb = chromadb.PersistentClient(vdb_path)
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2"
    )
    embedding_func = sentence_transformer_ef
    vectors = vdb.get_or_create_collection(
        name="arxiver", embedding_function=embedding_func
    )

    res = vectors.get(ids=[paper_id], limit=1, include=["embeddings"])
    # print(res)
    # print(f'{res["ids"][0]} {res["embeddings"][0]}')
    return res["embeddings"][0]

In [41]:
articles = get_data(db)
X_article = []
y_article = []
for article in articles:
    # print(
    #     f'{article["paper_id"]}, {article["interested"]}\n{article["concise_summary"]}'
    # )
    X_article.append(get_embedding(article["paper_id"]))
    y_article.append(article["interested"])

# print(X_article[:3])
# print(y_article[:3])

X = np.array(X_article)
y = np.array(y_article)

# print(X[:1])
# print(y[:1])
print(X.shape, y.shape)

Got 3003.
(3003, 384) (3003,)


In [42]:
print(len(X[0]))

384


In [43]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [44]:
def build_model(hp):
    model = Sequential()
    model.add(
        Dense(
            units=hp.Int("units1", min_value=32, max_value=512, step=32),
            activation="relu",
            input_shape=(X_train.shape[1],),
        )
    )
    model.add(
        Dropout(rate=hp.Float("dropout1", min_value=0.0, max_value=0.5, step=0.1))
    )
    model.add(
        Dense(
            units=hp.Int("units2", min_value=32, max_value=256, step=32),
            activation="relu",
        )
    )
    model.add(
        Dropout(rate=hp.Float("dropout2", min_value=0.0, max_value=0.5, step=0.1))
    )
    model.add(Dense(1, activation="sigmoid"))

    # Compile model
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [45]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=80,
    executions_per_trial=2,
    directory="my_dir",
    project_name="arxiv_article_embeddings",
)

Reloading Tuner from my_dir/arxiv_article_embeddings/tuner0.json


In [46]:
from tensorflow.keras.callbacks import EarlyStopping

stop_early = EarlyStopping(monitor="val_loss", patience=5)

tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [47]:
# Build the model with the optimal hyperparameters and train it on the data
new_model = tuner.hypermodel.build(best_hps)
new_model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
61/61 [==============================] - 1s 4ms/step - loss: 0.2530 - accuracy: 0.9485 - val_loss: 0.1820 - val_accuracy: 0.9480
Epoch 2/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1673 - accuracy: 0.9500 - val_loss: 0.1870 - val_accuracy: 0.9480
Epoch 3/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1453 - accuracy: 0.9500 - val_loss: 0.2004 - val_accuracy: 0.9480
Epoch 4/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1232 - accuracy: 0.9500 - val_loss: 0.2068 - val_accuracy: 0.9480
Epoch 5/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1033 - accuracy: 0.9500 - val_loss: 0.2352 - val_accuracy: 0.9480
Epoch 6/50
61/61 [==============================] - 0s 3ms/step - loss: 0.0810 - accuracy: 0.9526 - val_loss: 0.2562 - val_accuracy: 0.9459
Epoch 7/50
61/61 [==============================] - 0s 3ms/step - loss: 0.0592 - accuracy: 0.9724 - val_loss: 0.3234 - val_accuracy: 0.9397
Epoch 8/50
61/61 [==

In [48]:
best_hps.values

{'units1': 320,
 'dropout1': 0.0,
 'units2': 224,
 'dropout2': 0.4,
 'learning_rate': 0.001}

In [49]:
# # best_hps.values @ 20 iter

# {'units1': 320,
#  'dropout1': 0.0,
#  'units2': 224,
#  'dropout2': 0.4,
#  'learning_rate': 0.001}

In [50]:
# Build the model with the optimal hyperparameters but with matching input units
better_hps = {
    "units1": 384,
    "dropout1": 0.0,
    "units2": 224,
    "dropout2": 0.4,
    "learning_rate": 0.001,
}
better_model = tuner.hypermodel.build(best_hps)
better_model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
61/61 [==============================] - 1s 4ms/step - loss: 0.2514 - accuracy: 0.9427 - val_loss: 0.1829 - val_accuracy: 0.9480
Epoch 2/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1646 - accuracy: 0.9500 - val_loss: 0.1913 - val_accuracy: 0.9480
Epoch 3/50
61/61 [==============================] - 0s 3ms/step - loss: 0.1392 - accuracy: 0.9500 - val_loss: 0.2144 - val_accuracy: 0.9480
Epoch 4/50
61/61 [==============================] - 0s 2ms/step - loss: 0.1213 - accuracy: 0.9500 - val_loss: 0.2204 - val_accuracy: 0.9480
Epoch 5/50
61/61 [==============================] - 0s 3ms/step - loss: 0.0978 - accuracy: 0.9500 - val_loss: 0.2418 - val_accuracy: 0.9480
Epoch 6/50
61/61 [==============================] - 0s 3ms/step - loss: 0.0759 - accuracy: 0.9563 - val_loss: 0.2656 - val_accuracy: 0.9439
Epoch 7/50
61/61 [==============================] - 0s 3ms/step - loss: 0.0538 - accuracy: 0.9818 - val_loss: 0.3165 - val_accuracy: 0.9376
Epoch 8/50
61/61 [==